# Tutorial to generate checkerbord cell simulation with an artbitray expression

In [13]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
import skimage
from sklearn.utils.random import sample_without_replacement
from scipy.sparse import csr_array

In [14]:
import sys


import simtissue
from simtissue.arbitrary_shape import checkerboard_mask, generate_ellipse, add_sphere_nuclei

from simtissue.plot import plot_contour
from simtissue.control_expression import create_fix_profile, simulate_arbritrary_expression
from simtissue.utils import get_dict_coord_map

### 1° Generate checkerboard 
The first step is to generate a cells as checkerboard of the needed size

In [15]:
mask_cyto = checkerboard_mask(shape = [1, 1000, 1100],
                         cube_square_size = 100)
plot_contour(mask_cyto = mask_cyto, figsize = (5,5))

### 2) Add nuclei in the checker board
The number and position of the cell without nuclei can be parametrize with argument ``list_nuclei`` in ``add_sphere_nuclei``

In [16]:

## leave 20% of the cells without cytoplasm
list_nuclei_unique = np.unique(mask_cyto)
index  = sample_without_replacement(len(list_nuclei_unique), int(0.80 * len(list_nuclei_unique)))
list_nuclei = list_nuclei_unique[index]
mask_nuclei = add_sphere_nuclei(mask_cyto = mask_cyto, 
                     cube_square_size  = 100,
                     nuclei_radius = 25,
                     list_nuclei = list_nuclei)

plot_contour(mask_cyto = mask_cyto, mask_nuclei = mask_nuclei, figsize = (5,5))


### 3) Simulation of arbitrary cell expression

-we  simulate a cell-expression matrix containing the expression profile we want to map in the checkerboard

In [17]:
### generate_anndata

## RNA profile of each cell type 
genes_list_to_simulate = ['A', 'B']
dict_profile = {'typeA': [50, 0],
                  'typeB': [0, 200]
             }

## cell type of each cell index
cell_index_typeA = [i for i in range(1, 111) if i % 2 == 0]
cell_index_typeB = [i for i in range(1, 111) if i % 2 == 1]



cell_type_distribution={'typeA': cell_index_typeA,
                        'typeB': cell_index_typeB,}

image_name = "image0"

## get the anndata with simulate_arbritrary_expression()
anndata = simulate_arbritrary_expression(dict_profile,
                cell_type_distribution,
                mask_cyto = mask_cyto,
                genes_list_to_simulate = ['A', 'B'],
                image_name = "image0"
        )
anndata

### Plot of final simulation

In [18]:
plot_contour(mask_cyto = mask_cyto,
                 mask_nuclei = mask_nuclei,
                 figsize = (15,15),
                anndata = anndata,
                dico_color = {"A" : "r", "B": "b"},
                spot_size = 3)

### Save and load simulation

In [19]:
import pickle
from pathlib import Path
filename = "simulated_image.h5ad"
with open(filename, 'wb') as handle:
    pickle.dump(anndata, handle, -1)

In [20]:
file_pi2 = open(filename, 'rb')
anndata = pickle.load(file_pi2)
anndata